## Compute Grad-CAM

In [1]:
import tensorflow as tf

import sys
sys.path.append('/home/ubuntu/store/explanations/')

In [2]:
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from PIL import Image as Img

# For Grad-CAM
import xgradcam
from xgradcam import get_img_array
from xgradcam import make_gradcam_heatmap
from xgradcam import save_and_display_gradcam

from pathlib import Path
import os
import glob
import matplotlib.cm as cm 
from numpy import save
from numpy import load
from skimage.transform import resize
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator, DirectoryIterator

In [4]:
def save_and_display_gradcam(img_array, heatmap, cam_path='', alpha=0.4):
    # Load the original image
    img = img_array[0]
    # Rescale heatmap to a range 0-255    
    heatmap = np.uint8(255 * heatmap)
    
    # Resize heatmap to save 
    heatmap_resized = resize(heatmap, (300, 400))
    
    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    return superimposed_img, heatmap_resized

## Visualizations for EfficientNet

In [5]:
results_path = "/home/ubuntu/store/efficientnet-final/visualisation/gradcam/"
model_names = glob.glob("/home/ubuntu/store/efficientnet-final/*.h5")

for model_path in model_names:
    model_name = Path(model_path).stem
    model_results_path = os.path.join(results_path, model_name) 
    # create dir for each model
    if not os.path.exists(model_results_path):
        os.mkdir(model_results_path)
    
    model = load_model(Path(model_path))
    last_conv_layer_name = 'top_activation'
    model.layers[-1].activation = None

    data_generator = ImageDataGenerator(
        fill_mode='nearest',
        preprocessing_function=efficientnet_preprocessing
    )
    data_iterator = data_generator.flow_from_directory(
        '/home/ubuntu/store/DermX-test-set/test', 
        batch_size=1, 
        target_size=(300, 400),
        class_mode='categorical',
        follow_links=True,
        interpolation='bilinear',
        shuffle=False
    )
    class_indices = data_iterator.class_indices
    
    for (img_array, label), img_path in zip(data_iterator, data_iterator.filepaths):
        img_name = Path(img_path).stem
        preds = model.predict(img_array)
        for category in class_indices:
            print(model_name, img_name, category)
            heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name,class_indices[category])
            imposed,heat_map_resized = save_and_display_gradcam(img_array, heatmap)
            # save files
            save(model_results_path + '/' + img_name + '_' +  category + '.npy', heat_map_resized)
            imposed.save(model_results_path + '/' + img_name + '_' +  category + '.png')
            